In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [32]:
import pandas as pd

raw_csv_data = pd.read_csv('/content/Audiobooks_data.csv',delimiter=',')
unscaled_input_all=raw_csv_data.iloc[:,1:-1]
target_all=raw_csv_data.iloc[:,-1]

In [ ]:
#Balance the data set

In [33]:
num_one_targets=int(np.sum(target_all))
zero_targets_counter=0
indices_to_remove=[]

In [34]:
for i in range(target_all.shape[0]):
  if target_all[i]==0:
    zero_targets_counter+=1
    if zero_targets_counter>num_one_targets:
      indices_to_remove.append(i)

In [43]:
unscled_inputs_equal_priors=unscaled_input_all.drop(indices_to_remove,axis=0)
target_equal_priors=target_all.drop(indices_to_remove,axis=0).reset_index(drop=True)

In [36]:
#Standardize the inputs
scaled_inputs=StandardScaler().fit_transform(unscled_inputs_equal_priors)

In [40]:
#shuffle the data
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

In [45]:
shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=target_equal_priors[shuffled_indices]

In [46]:
sample_count=shuffled_inputs.shape[0]
train_sample_count=int(0.8*sample_count)
validation_sample_count=int(0.1*sample_count)
test_sample_count=sample_count-train_sample_count-validation_sample_count

In [47]:

train_inputs=shuffled_inputs[:train_sample_count]
train_targets=shuffled_targets[:train_sample_count]

In [49]:
validattion_inputs=shuffled_inputs[train_sample_count:train_sample_count+validation_sample_count]
validation_targets=shuffled_targets[train_sample_count:train_sample_count+validation_sample_count]

In [50]:
test_inputs=shuffled_inputs[train_sample_count+validation_sample_count:]
test_targets=shuffled_targets[train_sample_count+validation_sample_count:]

In [51]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validattion_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

In [59]:
npz=np.load('Audiobooks_data_train.npz')
train_inputs=npz['inputs'].astype(float)
train_targets=npz['targets'].astype(np.int32)

In [60]:
npz=np.load('Audiobooks_data_validation.npz')
validation_inputs=npz['inputs'].astype(float)
validation_targets=npz['targets'].astype(np.int32)

validation_test=np.load('Audiobooks_data_test.npz')
test_inputs=validation_test['inputs'].astype(float)
test_targets=validation_test['targets'].astype(np.int32)

In [69]:
input_size=10
output_size=2
hidden_layer_size=50
model=tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size,activation='relu', input_shape=(input_size,)),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax')
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [70]:
batchsize=100
max_epochs=100
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_inputs,
          train_targets,
          batch_size=batchsize,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs,validation_targets),
          verbose=2)


Epoch 1/100
36/36 - 1s - 37ms/step - accuracy: 0.7047 - loss: 0.5564 - val_accuracy: 0.7852 - val_loss: 0.4711
Epoch 2/100
36/36 - 0s - 13ms/step - accuracy: 0.7703 - loss: 0.4479 - val_accuracy: 0.8009 - val_loss: 0.4090
Epoch 3/100
36/36 - 0s - 9ms/step - accuracy: 0.7896 - loss: 0.4055 - val_accuracy: 0.8054 - val_loss: 0.3841
Epoch 4/100
36/36 - 0s - 8ms/step - accuracy: 0.7969 - loss: 0.3837 - val_accuracy: 0.8233 - val_loss: 0.3642
Epoch 5/100
36/36 - 0s - 9ms/step - accuracy: 0.7999 - loss: 0.3705 - val_accuracy: 0.8188 - val_loss: 0.3545
Epoch 6/100
36/36 - 0s - 4ms/step - accuracy: 0.8108 - loss: 0.3605 - val_accuracy: 0.8233 - val_loss: 0.3425
Epoch 7/100
36/36 - 0s - 8ms/step - accuracy: 0.8106 - loss: 0.3556 - val_accuracy: 0.8031 - val_loss: 0.3555
Epoch 8/100
36/36 - 0s - 4ms/step - accuracy: 0.8066 - loss: 0.3509 - val_accuracy: 0.8121 - val_loss: 0.3435


In [71]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7897 - loss: 0.3601 
